In [26]:
# -*- coding: UTF-8 -*-
from tempfile import tempdir
from tkinter import *
from tkinter import scrolledtext, filedialog
from unittest import expectedFailure
from selenium import webdriver
from time import sleep
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time, threading, sys, datetime, re, os, csv
import pandas as pd
import codecs
import math
import glob
import re
import configparser

"""""""""""""""""""""""""""""""""
初期値
"""""""""""""""""""""""""""""""""
DefID       = ''      # ログインIDのデフォルト値
DefPW       = ''      # ログインパスワードのデフォルト値
CONFIG      = 'config.ini' 

"""""""""""""""""""""""""""""""""
初期化
"""""""""""""""""""""""""""""""""
encode_type = {
    'utf_8'       : 'utf_8',
    'shift_jis'   : 'shift_jis'
}

class BuymaSell():
    """""""""""""""""""""""""""""""""
    基本処理
    """""""""""""""""""""""""""""""""
    def __init__(self, tkinterController):
        #encordingの取得
        enc_no = tkinterController.var.get()
        enc_type = 'shift_jis' if enc_no == 0 else 'utf_8'
        
        #出品情報CSV読み込み
         seller_csv = tkinterController.entry3.get()
         table_input = self.read_csv(seller_csv, enc_type)

        #出品情報XLSX読み込み
        seller_xlsx = tkinterController.entry3.get()
        table_input = self.read_xlsx(seller_xlsx, enc_type)

        #エンコーディングの読み直し ---------------------------------------------------------------
        #読み込んだCSVが文字化けしてないか確認
        try:
            temp    =   table_inputa
            is_enc_err  = False     #※フォルダ名を参照できた場合はEncordingエラーなし
        except:
            is_enc_err  = True      #※フォルダ名を参照できなかった場合はエラーとなるので、exceptに入る。Encordingエラーありと記録

        #もしエンコード違いで文字化けしていたら、エンコードを変更して読み直す        
        if is_enc_err:
            tkinterController.put_simple('情報：エンコーディングが違うので自動で変更して継続しています\n','info')
            enc_type = 'shift_jis' if enc_no == 1 else 'utf_8'  #エンコーディングを入れ替える
            table_input = self.read_xlsx(seller_xlsx, enc_type)

        #----------------------------------------------------------------------------------------

        #下書き/出品の切り替え
        exh_no = tkinterController.var2.get()
        exh_type = 'draft' if exh_no == 0 else 'official'

        # 出品モード情報
        if exh_type == 'draft':
            tkinterController.put_simple('下書きモードで開始\n','info')
        else:
            tkinterController.put_simple('出品モードで開始\n','info')

        #Chrome起動
        driver = self.run_driver()

        #ログイン
        is_login_succeed = self.buyma_login(driver, tkinterController)

        #出品指定数分を出品する　※ログイン失敗していたら終了する
        if is_login_succeed:
            for post_no in range(100):
                tkinterController.put('No. ' + str(post_no + 1) + '-------------------', 'info')
                self.input_sell_object(driver, table_input.loc[post_no,:], tkinterController, exh_type)

        #driverを閉じる
        driver.close()

        #終了メッセージ
        tkinterController.put('終了', 'info')

    """""""""""""""""""""""""""""""""
    関数：エクセル(.csv)読み込み
    """""""""""""""""""""""""""""""""
    def read_csv(self, input_file, enc):
        with codecs.open(input_file, "r", encode_type[enc], "ignore") as file:
            talbe = pd.read_table(file, delimiter=",")
        
        return talbe

    """""""""""""""""""""""""""""""""
    関数：エクセル(.xlsx)読み込み
    """""""""""""""""""""""""""""""""
    def read_xlsx(self, input_file, enc):
        input_book  = pd.ExcelFile(input_file)
        table       = input_book.parse()
        # with codecs.open(input_file, "r", encode_type[enc], "ignore") as file:
        #     talbe = pd.read_table(file, delimiter=",")
        
        return table

    """""""""""""""""""""""""""""""""
    関数：WebDriverをOpen
    """""""""""""""""""""""""""""""""
    def run_driver(self): #return -> driver
        
        #　ヘッドレスモードでブラウザを起動
        options = Options()
        options.add_argument("start-maximized")
        options.add_argument("enable-automation")
        #options.add_argument("--headless")
        options.add_argument("--no-sandbox")
        options.add_argument("--disable-infobars")
        options.add_argument('--disable-extensions')
        options.add_argument("--disable-dev-shm-usage")
        options.add_argument("--disable-browser-side-navigation")
        options.add_argument("--disable-gpu")
        options.add_argument('--ignore-certificate-errors')
        options.add_argument('--ignore-ssl-errors')
        
        # ブラウザーを起動
        driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)
            
        return driver

    """""""""""""""""""""""""""""""""
    関数：ログイン
    """""""""""""""""""""""""""""""""
    def buyma_login (self, driver, tkinterController):

        #指定のWebサイトに移動
        driver.get('https://www.buyma.com/login/')
        
        #ログイン userid
        LOGIN_ID = tkinterController.entry1.get()
        userid = driver.find_element_by_id('txtLoginId')
        userid.send_keys(LOGIN_ID)
        
        #ログイン passwd
        LOGIN_PW = tkinterController.entry2.get()
        passwd = driver.find_element_by_id('txtLoginPass')
        passwd.send_keys(LOGIN_PW)

        #ログインボタンを押す
        bt_login = driver.find_element_by_id('login_do')
        driver.execute_script("arguments[0].scrollIntoView(true);", bt_login)
        bt_login.click()
        sleep(3)
        
        #画面最大化
        driver.maximize_window()

        #ログインが成功したか確認（現在のページがloginで泣ければ成功とみなす）
        if 'login' in driver.current_url:
            is_login_successed = False
            tkinterController.put('ログインに失敗しました','info')
        else:
            is_login_successed = True

        return is_login_successed


    """""""""""""""""""""""""""""""""
    サブ関数：プルダウン選択型のセレクト
    """""""""""""""""""""""""""""""""
    def solo_pulldown_select(self, driver, el, select_word, num):
        try:
            # セレクトされたかを判定する変数
            is_selected = False

            # 開きたいプルダウンまでスクロール
            pull_down = el.find_elements_by_css_selector('.Select-control')
            driver.execute_script("arguments[0].scrollIntoView(true);", pull_down[num])
            #sleep(0.3)

            # プルダウンを開く
            pull_down[num].click()
            #sleep(0.5)
            sleep(0.1)

            # プルダウンの選択肢を順番に確認
            selects = el.find_elements_by_css_selector('.Select-option')
            for sl in selects:

                # 該当の選択肢の場合はクリック
                if sl.get_attribute('aria-label') == select_word:
                    sl.click()
                    is_selected = True
                    break

            # セレクトされていなかったらクリックしてプルダウンを閉じる
            if is_selected == False:
                pull_down[num].click()

            #sleep(0.5)
        except:
            pass

    """""""""""""""""""""""""""""""""
    サブ関数：画像フォルダの選択
    """""""""""""""""""""""""""""""""
    def pic_select(self, el, pic_folder, driver):
        try:
            # フォルダ内のファイル名を取得し、一括アップロード用にテキスト編集      
            files = sorted(glob.glob(pic_folder + '/*'), key=self.natural_keys)
            files = '\n'.join(files)

            # カラムのブロック群を取得
            elements = driver.find_elements_by_css_selector('.bmm-c-panel__item')
            for el in elements:
                # カラムのタイトルを取得し、該当カラムかどうかを確認
                web_title = el.find_element_by_css_selector('.bmm-c-summary__ttl')

                # 1. フォルダ名
                if web_title.text == '商品画像':                
                    el.find_element_by_tag_name('input').send_keys(files)
                    sleep(0.5)
        except:
            pass

    """""""""""""""""""""""""""""""""
    関数：画像のソート用
    """""""""""""""""""""""""""""""""
    def atoi(self, text):
        return int(text) if text.isdigit() else text

    def natural_keys(self, text):
        return [ self.atoi(c) for c in re.split(r'(\d+)', text) ]


    """""""""""""""""""""""""""""""""
    関数：出品（１つ）
    """""""""""""""""""""""""""""""""
    def input_sell_object(self, driver, table_input_one_row, tkinterController, exh_type):

        #出品ページへ移動
        driver.get('https://www.buyma.com/my/sell/new?tab=b')

        #ページ移動アラート対応
        try:
            WebDriverWait(driver, 1).until(EC.alert_is_present())    
            alert = driver.switch_to.alert
            alert.accept()
            print("alert accepted")
        except:
            print("no alert")

        #待機処理
        sleep(5)

        #「カタログ出品」のポップアップの「スキップ」を押下
        try:
            driver.find_element_by_class_name('driver-close-btn').click()
        except:
            pass
        
        """ [1週目] 最初から出ているフィールド """
        # 出品画面のカラムを順番に確認
        elements = driver.find_elements_by_css_selector('.bmm-c-panel__item')
        for el in elements:

            # bmm-c-summary__ttlがないカラムができたのでエラー時は飛ばす処理を追加（2023.2.12）
            try:
                # カラムのタイトルを取得し、該当カラムかどうかを確認
                web_title = el.find_element_by_css_selector('.bmm-c-summary__ttl')

                print(web_title.text)

                # 1. フォルダ名
                if web_title.text == '商品画像':
                    try:
                        self.pic_select(el, table_input_one_row['※フォルダ名'], driver)
                        print('1. フォルダ名 done')
                    except:
                        print('1. フォルダ名 error')

                # 2. 商品名
                elif web_title.text == '商品名':
                    try:
                        goods_name = el.find_element_by_css_selector('.bmm-c-text-field')
                        goods_name.send_keys(str(table_input_one_row['※商品名'][:60]))
                        print('2. 商品名 done')
                    except:
                        print('2. 商品名 error')

                # 3. 商品コメント
                elif web_title.text == '商品コメント':
                    try:
                        goods_comment = el.find_element_by_css_selector('.bmm-c-textarea')
                        goods_comment.send_keys(table_input_one_row['※商品コメント'])
                        print('3. 商品コメント done')
                    except:
                        print('3. 商品コメント error')

                # 4~6. カテゴリ 
                elif web_title.text == 'カテゴリ':
                    try:

                        # カテゴリ名の一覧をリストに入れる
                        select_words = [
                            table_input_one_row['※カテゴリ ー 第１階層'],
                            table_input_one_row['カテゴリ ー 第２階層'],
                            table_input_one_row['カテゴリ ー 第３階層']
                        ]
                        # カテゴリ1,2,3階層を回す
                        for i, word in enumerate(select_words):
                            #値が無ければ終了
                            if word == '':
                                break
                            self.solo_pulldown_select(driver, el, word, i)

                            #入れた値が一致しているか確認
                            pull_down = el.find_elements_by_css_selector('.Select-control')
                            loop_num = 0
                            while pull_down[i].text != word and loop_num < 5:
                                self.solo_pulldown_select(driver, el, word, i)
                                loop_num = loop_num + 1
                                #tkinterController.put('再投入 : ' + str(loop_num), "info")

                        print('4~6. カテゴリ done')
                    except:
                        print('4~6. カテゴリ error')

                # 7. ブランド
                elif web_title.text == 'ブランド':
                    try:
                        if str(table_input_one_row['※ブランド']) =='nan':
                            el.find_element_by_css_selector('.bmm-c-checkbox__body').click()
                        else:
                            brand = el.find_element_by_css_selector('.bmm-c-text-field')
                            brand.send_keys(str(table_input_one_row['※ブランド']))
                            sleep(2)
                        print('7. ブランド done')
                    except:
                        print('7. ブランド error')

                # 8. シーズン
                elif web_title.text == 'シーズン':
                    try:
                        self.solo_pulldown_select(driver, el, table_input_one_row['シーズン'], 0)
                        self.solo_pulldown_select(driver, el, table_input_one_row['シーズン'], 0) #2回やるといけるから暫定対応
                        print('8. シーズン done')
                    except:
                        print('8. シーズン error')

                # 9. テーマ
                elif web_title.text == 'テーマ':
                    try:
                        self.solo_pulldown_select(driver, el, table_input_one_row['テーマ'], 0)
                        self.solo_pulldown_select(driver, el, table_input_one_row['テーマ'], 0) #2回やるといけるから暫定対応
                        print('9. テーマ done')
                    except:
                        print('9. テーマ error')


                # （10. タグ）
                # カテゴリ第三要素入力後しかこのフィールドが出てこないので、このループ後に入れる
                # ここでは色とサイズの数の掛けた数を記録

                # 11~17. 色・サイズ
                elif web_title.text == '色・サイズ':
                    try:

                        # 色とサイズのListを取得
                        color_type_ls = list(table_input_one_row['色 ー 色の系統'].split('/'))      if type(table_input_one_row['色 ー 色の系統']) == str else ''
                        color_name_ls = list(table_input_one_row['色 ー 色名'].split('/'))          if type(table_input_one_row['色 ー 色名']) == str else ''
                        size_name_ls  = list(str(table_input_one_row['サイズ ー サイズ名']).split('/'))
                        size_JPref_ls = list(str(table_input_one_row['サイズ ー 参考日本サイズ']).split('/'))

                        # 「色 ー 色の系統」の数だけ登録作業を実施
                        for i in range(len(color_type_ls)):
                            # 「色 ー 色の系統」が2つ以上設定されていたら「＋新しい色を追加」をクリックして色増やす
                            if i >= 1:
                                el.find_element_by_css_selector('.bmm-c-ico.bmm-c-ico-plus').click()

                            # 色指定ありのときのみ
                            if color_type_ls[i] != '色指定なし':
                                # 11. 色 ー 色の系統
                                self.solo_pulldown_select(driver, el, color_type_ls[i], i)

                                # 12. 色 ー 色名
                                color_name = el.find_elements_by_css_selector('.bmm-c-text-field')
                                color_name[i].send_keys(color_name_ls[i])

                        # サイズタブをクリック
                        size_tab = el.find_element_by_id('react-tabs-2')
                        size_tab.click()
                        sleep(0.3)

                        # 13. サイズ ー バリエーション有無
                        self.solo_pulldown_select(driver, el, table_input_one_row['サイズ ー バリエーション有無'], 0)

                        # 14. サイズ ー 単位
                        self.solo_pulldown_select(driver, el, table_input_one_row['サイズ ー 単位'], 1)

                        # 「サイズ ー サイズ名」の数だけ登録作業を実施
                        for i in range(len(size_name_ls)):
                            # 「サイズ ー サイズ名」が2つ以上設定されていたら「＋新しいサイズを追加」をクリックして色増やす
                            if i >= 1:
                                el.find_element_by_css_selector('.bmm-c-ico.bmm-c-ico-plus').click()

                            # 「バリエーションあり」の場合
                            if table_input_one_row['サイズ ー バリエーション有無'] == 'バリエーションあり':
                                # 15. サイズ ー サイズ名
                                color_name = el.find_elements_by_css_selector('.bmm-c-text-field')
                                color_name[i].send_keys(size_name_ls[i])

                                # 16. サイズ ー 参考日本サイズ
                                self.solo_pulldown_select(driver, el, size_JPref_ls[i], i+2)

                            # 「バリエーションなし」の場合
                            elif table_input_one_row['サイズ ー バリエーション有無'] == 'バリエーションなし':
                                # 16. サイズ ー 参考日本サイズ
                                self.solo_pulldown_select(driver, el, size_JPref_ls[0], 2)

                        # 17. 色・サイズ補足情報
                        # 情報が入っているときだけ入力
                        if type(table_input_one_row['色・サイズ補足情報']) == str:
                            color_comment = el.find_element_by_css_selector('.bmm-c-textarea')
                            color_comment.send_keys(str(table_input_one_row['色・サイズ補足情報']))

                        print('11~17. 色・サイズ done')
                    except:
                        print('11~17. 色・サイズ error')

                # 19~21. 配送方法
                elif web_title.text == '配送方法':
                    try:
                        # クリックしたい要素までスクロール
                        check_bt = el.find_elements_by_css_selector('.bmm-c-form-table__icon-cell')
                        driver.execute_script("arguments[0].scrollIntoView(true);", check_bt[0])

                        # 配送方法のListを取得
                        shipping_ls = list(table_input_one_row['※配送方法'].split('/'))

                        # 配達方法の表要素取得
                        el_table = el.find_element_by_css_selector('.bmm-c-form-table__body')

                        # 各配送方法をテキスト検索してクリック
                        for shipping in shipping_ls:
                            for tr in el_table.find_elements_by_tag_name('tr'):
                                try:
                                    # 配送方法の列を指定
                                    temp_ship = tr.find_elements_by_tag_name('td')[1]

                                    # 配送方法のみ取得（補足説明部分は除去）
                                    temp_ship_wo_supp = temp_ship.find_element_by_tag_name('p').text

                                    # 配送方法が合致したらクリック
                                    if temp_ship_wo_supp == shipping:
                                        temp_ship.click()
                                        break
                                except:
                                    pass
                                
                        print('19~21. 配送方法 done')
                    except:
                        print('19~21. 配送方法 error')

                # 22. 購入期限
                elif web_title.text == '購入期限(日本時間)':
                    try:
                        #追加（readonlyを削除）
                        driver.execute_script("document.querySelector('.bmm-c-text-field.sell-term').removeAttribute('readonly');")
                        #日付書き込み
                        limit_date = el.find_element_by_css_selector('.bmm-c-text-field.sell-term')
                        limit_date.send_keys(Keys.COMMAND,"a")  #macの場合に対応できるように、command+a
                        limit_date.send_keys(Keys.CONTROL,"a")  #windowsの場合に対応できるように、control+a
                        limit_date.send_keys(table_input_one_row['※購入期限'])
                        limit_date.send_keys(Keys.ENTER)

                        print('22. 購入期限 done')
                    except:
                        print('22. 購入期限 error')                    

                # 23~26. 買付地
                elif web_title.text == '買付地':
                    try:
                        # 23. ※買付地 ー 国内外
                        # クリックしたい要素までスクロール
                        radio_bt = el.find_elements_by_css_selector('.bmm-c-radio__body')
                        driver.execute_script("arguments[0].scrollIntoView(true);", radio_bt[0])

                        if (table_input_one_row['※買付地 ー 国内外'] == '海外'):
                            radio_bt[1].click()
                        else:
                            radio_bt[0].click()

                        # 24. 買付地 ー 地域
                        self.solo_pulldown_select(driver, el, table_input_one_row['買付地 ー 地域'], 0)

                        # 25. 買付地 ー 国 -> [固定]　※注意、国内選択時はこのフィールドなし
                        self.solo_pulldown_select(driver, el, table_input_one_row['買付地 ー 国'], 1)

                        # 26. 買付地 ー 都市
                        self.solo_pulldown_select(driver, el, table_input_one_row['買付地 ー 都市'], 2)

                        print('23~26. 買付地 done')
                    except:
                        print('23~26. 買付地 error')

                # 27. 買付先ショップ名
                elif web_title.text == '買付先ショップ名':
                    try:
                        shop_name = el.find_element_by_css_selector('.bmm-c-text-field')
                        shop_name.send_keys(str(table_input_one_row['買付先ショップ名']) if str(table_input_one_row['買付先ショップ名']) != 'nan' else '')
                        print('27. 買付先ショップ名 done')
                    except:
                        print('27. 買付先ショップ名 error')

                # 28~31. 発送地
                elif web_title.text == '発送地':
                    try:
                        # 28. ※発送地 ー 国内外
                        # クリックしたい要素までスクロール
                        radio_bt = el.find_elements_by_css_selector('.bmm-c-radio__body')
                        driver.execute_script("arguments[0].scrollIntoView(true);", radio_bt[0])

                        if (table_input_one_row['※発送地 ー 国内外'] == '海外'):
                            radio_bt[1].click()
                        else:
                            radio_bt[0].click()

                        # 29. 発送地 ー 地域
                        self.solo_pulldown_select(driver, el, table_input_one_row['発送地 ー 地域'], 0)

                        # 30. 発送地 ー 国 -> [固定]　※注意、国内選択時はこのフィールドなし
                        self.solo_pulldown_select(driver, el, table_input_one_row['発送地 ー 国'], 1)

                        # 31. 発送地 ー 都市
                        self.solo_pulldown_select(driver, el, table_input_one_row['発送地 ー 都市'], 2)

                        print('28~31. 発送地 done')
                    except:
                        print('28~31. 発送地 error')

                # 32. 商品価格
                elif web_title.text == '商品価格':
                    try:
                        price = el.find_element_by_css_selector('.bmm-c-text-field.bmm-c-text-field--half-size-char')
                        price.send_keys(int(table_input_one_row['※商品価格']))
                        print('32. 商品価格 done')
                    except:
                        print('32. 商品価格 error')

                # 33~34. 参考価格/通常出品価格
                elif web_title.text == '参考価格/通常出品価格':
                    try:
                        # 33. 参考価格（ラジオボタン）
                        # クリックしたい要素までスクロール
                        radio_bt = el.find_elements_by_css_selector('.bmm-c-radio__body')
                        driver.execute_script("arguments[0].scrollIntoView(true);", radio_bt[1])

                        if table_input_one_row['参考価格/通常出品価格'] == '参考価格':
                            radio_bt[1].click()

                            # 34. 参考価格
                            price_ref = el.find_element_by_css_selector('.bmm-c-text-field.bmm-c-text-field--half-size-char')
                            price_ref.send_keys(int(table_input_one_row['参考価格']))

                        print('33~34. 参考価格/通常出品価格 done')
                    except:
                        print('33~34. 参考価格/通常出品価格 error')

                # 35~36. 関税
                elif web_title.text == '関税':
                    try:
                        # 関税込みならチェックボックスをON
                        if table_input_one_row['関税 ー 支払方法'] == '関税込み':
                            check_bt = el.find_element_by_css_selector('.bmm-c-checkbox__body')
                            check_bt.click()

                            # 関税負担先が購入者なら購入者側をチェック
                            if table_input_one_row['支払方法'] == '購入者':
                                # クリックしたい要素までスクロール
                                radio_bt = el.find_elements_by_css_selector('.bmm-c-radio__body')
                                driver.execute_script("arguments[0].scrollIntoView(true);", radio_bt[1])

                                #購入者側のラジオボタンをクリック
                                radio_bt[1].click()
                        print('35~36. 関税 done')
                    except:
                        print('35~36. 関税 error')

                # 37. 出品メモ
                elif web_title.text == '出品メモ':
                    try:
                        sales_memo = el.find_element_by_css_selector('.bmm-c-textarea')
                        sales_memo.send_keys(str(table_input_one_row['出品メモ']) if str(table_input_one_row['出品メモ']) != 'nan' else '')
                        print('37. 出品メモ done')
                    except:
                        print('37. 出品メモ error')

                # 38~40. 買付先メモ
                elif web_title.text == '買付先メモ':
                    try:
                        purchase_memo_fields = el.find_elements_by_css_selector('.bmm-c-text-field')
                        purchase_memo_fields[0].send_keys(str(table_input_one_row['買付先メモ ー 買付先']) if str(table_input_one_row['買付先メモ ー 買付先']) != 'nan' else '')
                        purchase_memo_fields[1].send_keys(str(table_input_one_row['買付先メモ ー 仕入先URL']) if str(table_input_one_row['買付先メモ ー 仕入先URL']) != 'nan' else '')
                        purchase_memo_fields[2].send_keys(str(table_input_one_row['買付先メモ ー 説明']) if str(table_input_one_row['買付先メモ ー 説明']) != 'nan' else '')
                        print('38~40. 買付先メモ done')
                    except:
                        print('38~40. 買付先メモ error')
            # bmm-c-summary__ttlがないカラムができたのでエラー時は飛ばす処理を追加（2023.2.12）
            except:
                pass

        """ [2週目] 値を入れると後から出現するフィールド """
        # 出品画面のカラムを順番に確認
        elements = driver.find_elements_by_css_selector('.bmm-c-panel__item')
        for el in elements:

            # bmm-c-summary__ttlがないカラムができたのでエラー時は飛ばす処理を追加（2023.2.12）
            try:
                # カラムのタイトルを取得し、該当カラムかどうかを確認
                web_title = el.find_element_by_css_selector('.bmm-c-summary__ttl')

                # 10. タグ
                if web_title.text == 'タグ':
                    try:
                        # 選択するタグ候補をlistで取得
                        tag_ls = list(table_input_one_row['タグ'].split('/'))      if type(table_input_one_row['タグ']) == str else ''

                        #タグが入っている場合のみ実行
                        if tag_ls != '':
                            # 「一覧からタグを選択」をクリック
                            add_tag = el.find_element_by_css_selector('.bmm-c-ico.bmm-c-ico-list-search')
                            add_tag.click()
                            sleep(1)

                            # タグのフィールドと、チェックボックスのリストを取得
                            tag_select_field = driver.find_element_by_css_selector('.bmm-c-modal__scrollable')
                            tags_in_select_field = tag_select_field.find_elements_by_css_selector('.bmm-c-checkbox__body')

                            # タグ候補分だけループさせる
                            for tag in tag_ls:
                                for tag_check_box in tags_in_select_field:
                                    if tag_check_box.text == tag:
                                        tag_check_box.click()
                                        break
                            
                            # 「選択したタグを設定」をクリック
                            define = driver.find_element_by_css_selector('.bmm-c-btn.bmm-c-btn--p.bmm-c-btn--l')
                            define.click()
                        print('10. タグ done')
                    except:
                        print('10. タグ error')

                # 18. 販売可否/在庫
                elif web_title.text == '販売可否/在庫':
                    try:
                        # 色とサイズのListを取得
                        #color_type_ls = list(table_input_one_row['色 ー 色の系統'].split('/'))      if type(table_input_one_row['色 ー 色の系統']) == str else ''
                        #size_name_ls  = list(str(table_input_one_row['サイズ ー サイズ名']).split('/'))
                        #Purch_ava_num = max(len(color_type_ls), 1) * max(len(size_JPref_ls), 1)
                        #Purch_ava_num = 2

                        stock = el.find_element_by_css_selector('.bmm-c-text-field.bmm-c-text-field--half-size-char')            
                        stock.send_keys(str(int(table_input_one_row['数量'])))
                        print('18. 販売可否/在庫 done')
                    except:
                        print('18. 販売可否/在庫 error')

                # 追加1. モデル
                elif web_title.text == 'モデル':
                    try:
                        self.solo_pulldown_select(driver, el, table_input_one_row['モデル'], 0)
                        print('追加1. モデル done')
                    except:
                        print('追加1. モデル error')

                # 追加2,3. 型番/識別メモ
                elif web_title.text == '型番/識別メモ':
                    try:
                        # 型番のListを取得
                        kata_type_ls = list(table_input_one_row['型番'].split('/'))      if type(table_input_one_row['型番']) == str else ''
                        shik_memo_ls = list(table_input_one_row['識別メモ'].split('/'))          if type(table_input_one_row['識別メモ']) == str else ''

                        # 「型番」の数だけ登録作業を実施
                        for i in range(len(kata_type_ls)):
                            # 「型番」が2つ以上設定されていたら「＋新しい色を追加」をクリックして色増やす
                            if i >= 1:
                                kata_add = el.find_element_by_css_selector('.bmm-c-ico.bmm-c-ico-plus')
                                driver.execute_script("arguments[0].scrollIntoView(true);", kata_add)
                                sleep(0.3)

                                kata_add.click()

                            text_fild = el.find_elements_by_css_selector('.bmm-c-text-field')
                            
                            # 追加2. 型番
                            if len(kata_type_ls) > 0:
                                text_fild[2*i].send_keys(kata_type_ls[i])

                            # 追加3. 識別メモ
                            if len(shik_memo_ls) > 0:
                                text_fild[2*i+1].send_keys(shik_memo_ls[i])
                        print('追加2,3. 型番・識別メモ done')
                    except:
                        print('追加2,3. 型番・識別メモ error')                    

            # bmm-c-summary__ttlがないカラムができたのでエラー時は飛ばす処理を追加（2023.2.12）
            except:
                pass

        try:
            if exh_type == 'official':
            #「入力内容を確認する」ボタンを押す
                save_exhibit_bt = driver.find_elements_by_css_selector('.bmm-c-btn--thick')[1]
                save_exhibit_bt.click()

            else:
                #「下書き保存する」ボタンを押す
                save_draft_bt = driver.find_elements_by_css_selector('.bmm-c-btn--thick')[0]
                save_draft_bt.click()

            #処理待機
            sleep(1.75)
        except:
            pass

        try:
            #出品エラーチェック
            error_message = driver.find_element_by_css_selector('.bmm-c-box--overall-alert').text
            tkinterController.put_simple('保存時エラー発生！！\n' + error_message, 'info')                
        except:
            try:
                # 正式出品の場合は「公開する」ボタンを押す
                if exh_type == 'official':
                    submit_bt = driver.find_element_by_css_selector('.bmm-c-btn.bmm-c-btn--p.bmm-c-btn--l')
                    submit_bt.click()

                # 保存完了メッセージ
                tkinterController.put_simple('保存完了', 'info')
            except:
                # 保存完了メッセージ
                tkinterController.put_simple('保存ボタンを押せなかった', 'info')





"""""""""""""""""""""""""""""""""
Tkinterのメインフレーム
"""""""""""""""""""""""""""""""""
class Application(Frame):
    def __init__(self, master):
        super().__init__(master)
        self.master.geometry("600x300")
        self.master.title("Buyma自動出品@Shirasu")
        self.th=threading.Thread(target=self.searchQuery)
        self.make()

    def make(self):
        # フレーム生成
        ctlfr = Frame(self.master)
        ctlfr.pack(anchor=W, padx=5)

        # ID/パスの取得
        id, passwd = self.read_config()

        # インプットファイルのデフォルト値取得
        filedef = self.get_def_inputfilepath()

        # ログインID
        label1 = Label(ctlfr, text="BUYMA ログインID",width=20)
        label1.grid(row=0, column=0, padx=5, pady=5)
        self.entry1 = Entry(ctlfr)
        self.entry1.insert(0, id)
        self.entry1.grid(row=0, column=1, padx=5, pady=5)
        self.entry1.focus()

        # パスワード
        label2 = Label(ctlfr, text="BUYMA パスワード")
        label2.grid(row=1, column=0, padx=5, pady=5)
        self.entry2 = Entry(ctlfr, show='*')
        self.entry2.insert(0, passwd)
        self.entry2.grid(row=1, column=1, padx=5, pady=5)
        self.entry2.focus()

        # ラジオボタン（エンコーディング）
        self.var = IntVar()
        self.var.set(0)
        self.rad1 = Radiobutton(ctlfr, text="shift_jis", value=0, variable = self.var)
        self.rad1.grid(row=0, column=2, padx=5, pady=5, sticky = W)
        self.rad2 = Radiobutton(ctlfr, text="utf_8", value=1, variable = self.var)
        self.rad2.grid(row=0, column=3, padx=1, pady=5, sticky = W)

        self.rad1["state"] = DISABLED   #エンコーディングは自動で選択
        self.rad2["state"] = DISABLED   #エンコーディングは自動で選択

        #入力ファイル開くボタン
        # self.btn2 = Button(ctlfr, text="入力ファイルを開く", command=self.inputFromCSV)
        self.btn2 = Button(ctlfr, text="入力ファイルを開く", command=self.inputFromXlsx)
        self.btn2.grid(row=3, column=0, padx=0, pady=0)
        self.entry3 = Entry(ctlfr)
        self.entry3.insert(0, filedef)
        self.entry3.grid(row=3, column=1, padx=5, pady=5)

        #ラジオボタン（下書き/出品切り替え）
        self.var2 = IntVar()
        self.var2.set(0)
        self.rad3 = Radiobutton(ctlfr, text="下書き", value=0, variable = self.var2)
        self.rad3.grid(row=1, column=2, padx=5, pady=5, sticky = W)
        self.rad4 = Radiobutton(ctlfr, text="出品", value=1, variable = self.var2)
        self.rad4.grid(row=1, column=3, padx=1, pady=5, sticky = W)

        #実行ボタン
        self.btn1 = Button(ctlfr, text="実行", command=self.start)
        self.btn1.grid(row=3, column=2, padx=5, pady=5)
        self.btn3 = Button(ctlfr, text="終了", command=self.master.quit)
        self.btn3.grid(row=3, column=3, padx=1, pady=5, sticky = W)

        # ログエリア
        self.logging_area = scrolledtext.ScrolledText(self.master, wrap=WORD)
        self.logging_area.pack(anchor=W, padx=5, fill=BOTH, expand=1, pady=5)
        self.logging_area.tag_config("info", foreground="black")
        self.logging_area.tag_config("warning", foreground="pink")
        self.logging_area.tag_config("error", background="yellow", foreground="red")

    def searchQuery(self):
        tb = BuymaSell(self)
    
    def start(self):
        if self.entry1.get() != "":
            self.btn1["state"] = DISABLED
            self.btn2["state"] = DISABLED
            self.rad1["state"] = DISABLED
            self.rad2["state"] = DISABLED
            self.rad3["state"] = DISABLED
            self.rad4["state"] = DISABLED
            self.entry1["state"] = DISABLED
            self.entry2["state"] = DISABLED
            self.entry3["state"] = DISABLED
            self.th.start()

    def put(self, text, tag):
        dt_now = datetime.datetime.now()
        dt     = str(dt_now.strftime('%Y.%m.%d.%H:%M:%S -- '))
        log    = dt + text + '\n'
        print(log)
        self.logging_area.insert(END, log, tag)
    
    def put_simple(self, text, tag):
        log    = text + '\n'
        print(log)
        self.logging_area.insert(END, log, tag)

    #完了時処理
    def finished(self):
        self.th = None
        self.th = threading.Thread(target=self.searchQuery)
        self.btn1["state"] = NORMAL
        self.entry1["state"] = NORMAL

    #CSVを開いてフィールドに格納
    def inputFromCSV(self):
        fTyp = [("CSV File", "*.csv")]    
        iDir = os.path.abspath(os.path.dirname(__file__))
        path = filedialog.askopenfilename(filetypes=fTyp, initialdir=iDir, defaultextension="csv")
        self.entry3.delete(0, END)
        self.entry3.insert(END, path)

    #XLSXを開いてフィールドに格納
    def inputFromXlsx(self):
        fTyp = [("XLSX File", "*.xlsx")]    
        iDir = os.path.abspath(os.path.dirname(__file__))
        path = filedialog.askopenfilename(filetypes=fTyp, initialdir=iDir, defaultextension="csv")
        self.entry3.delete(0, END)
        self.entry3.insert(END, path)

    # iniファイルの読み込み
    def read_config(self):
        config_ini = configparser.ConfigParser()
        config_ini_path = CONFIG

        # 指定したiniファイルが存在しない場合、エラー発生
        if not os.path.exists(config_ini_path):
            id, passwd = '',''

        config_ini.read(config_ini_path, encoding='utf-8')

        # iniの値取得
        read_default = config_ini['DEFAULT']
        id = read_default.get('id')
        passwd = read_default.get('passwd')

        return id, passwd
    
    # 初期インプットエクセルのパス
    def get_def_inputfilepath(self):

        try:
            input_file = glob.glob('*.xlsx')[0]
        except:
            input_file = ''

        return input_file


"""""""""""""""""""""""""""""""""
main
"""""""""""""""""""""""""""""""""
def main():
    win = Tk()
    app = Application(win)
    app.mainloop()
    
if __name__ == "__main__":
    main()

IndentationError: unexpected indent (865909824.py, line 46)

In [ ]:
from selenium.webdriver.common.by import By

In [ ]:
from time import sleep

In [ ]:
browser = webdriver.Chrome()

In [ ]:
url = "https://www.buyma.com/loginredirect/"
browser.get(url)

In [ ]:
elem_username = browser.find_element(By.ID, "txtLoginId")

In [ ]:
elem_username.send_keys("imanishi")

In [ ]:
elem_password = browser.find_element(By.ID, "password")

In [ ]:
elem_password.send_keys("kouhei")

In [ ]:
elem_login_btn = browser.find_element(By.ID, "login-btn")

In [ ]:
elem_login_btn.click()

In [ ]:
browser = webdriver.Chrome()
url = "https://scraping-for-beginner.herokuapp.com/login_page"
browser.get(url)
sleep(4)

elem_username = browser.find_element(By.ID, "username")
elem_username.send_keys("imanishi")

elem_password = browser.find_element(By.ID, "password")
elem_password.send_keys("kouhei")


In [ ]:

elem_login_btn = browser.find_element(By.ID,"login-btn")
elem_login_btn.click()

In [ ]:
!pip install Pillow

In [ ]:
image_dir_path = "/Users/nakaosatoshi/Desktop/画像"
# 画像ファイルを取得してアップロードする
for filename in os.listdir(image_dir_path):
    file_path = os.path.join(image_dir_path, filename)
    file_input = driver.find_element_by_xpath("//input[image/jpeg,image/gif,image/png']")
    file_input.send_keys(file_path)